In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import functools
import math
import tensorflow as tf
import tensorflow_datasets as tfds
from datetime import datetime
from tensorflow.python.ops import nn_ops
from tensorflow import keras
from fastcore.nb_imports import *
from fastcore.test import *
from fastcore.foundation import *

# Gabor Layer Math

$$G_\theta(x',y';\sigma,\gamma,\lambda,\psi):=e^{-\frac{(x'^2+\gamma^2y'^2)}{\sigma^2}}cos(\lambda x' + \psi)$$
$$x' = x cos\theta - y sin\theta$$
$$y' = x sin\theta + y cos\theta$$

In [ ]:
#export
class GaborLayer(keras.layers.Layer):
    def __init__(self, filters, kernel_size, orientations, learn_orientations=False,
                 use_bias=True, activation='relu', strides=(1, 1), padding='SAME', use_alphas=True, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.orientations = orientations
        self.learn_orientations = learn_orientations
        self.use_bias = use_bias
        self.strides = strides
        # can use strings as activation functions
        self.activation = keras.activations.get(activation)
        self.padding = padding
        self.rank = 2
        self.pi = tf.constant(math.pi, dtype=tf.float32)
        self.use_alphas = use_alphas

In [ ]:
#export
@patch
def create_xy_grid(self:GaborLayer):
    xmin, xmax = -1.0, 1.0
    ymin, ymax = -1.0, 1.0
    x_space = tf.linspace(xmin, xmax, self.kernel_size[0])
    y_space = tf.linspace(ymin, ymax, self.kernel_size[1])
    x, y = tf.meshgrid(y_space, x_space)
    
    return x, y

Test the outputs size of the meshgrid.

In [ ]:
kernel_size = (10, 10)
x, y = GaborLayer(filters=10, kernel_size=kernel_size, orientations=8).create_xy_grid()

test_eq(x.shape, kernel_size)
test_eq(y.shape, kernel_size)

kernel_size = (5, 10)
x, y = GaborLayer(filters=10, kernel_size=kernel_size, orientations=8).create_xy_grid()

test_eq(x.shape, kernel_size)
test_eq(y.shape, kernel_size)

In [ ]:
#export
@patch
def build(self:GaborLayer, batch_input_shape):
        
        self.gammas = self.add_weight(name='gammas', shape=[self.filters, 1, 1, 1], 
                                            initializer='ones', trainable=True)
        self.psis = self.add_weight(name='psis', shape=[self.filters, 1, 1, 1], 
                                   initializer='ones', trainable=True)
        self.sigmas = self.add_weight(name='sigmas', shape=[self.filters, 1, 1, 1], 
                                   initializer='ones', trainable=True)
        self.lambdas = self.add_weight(name='lambdas', shape=[self.filters, 1, 1, 1], 
                                   initializer='ones', trainable=True)
        
        if self.use_alphas:
            self.alphas = self.add_weight(name='alphas', shape=[self.filters * self.orientations, 1, 1], 
                                   initializer='ones', trainable=True)
        
                
        thetas = (tf.range(0, self.orientations, dtype=tf.float32) * 2 * self.pi) / self.orientations
        thetas = tf.reshape(thetas, (self.orientations, 1 ,1))
        self.thetas = tf.Variable(thetas, name='thetas', trainable=self.learn_orientations)
        
        x, y = self.create_xy_grid()
        
        x = x[None, :, :]
        y = y[None, :, :]
        
        sines = tf.sin(self.thetas)
        cosines = tf.cos(self.thetas)
        
        self.sines = sines
        self.cosines = cosines
        xprime = x * cosines - y * sines
        yprime = x * sines + y * cosines
        
        xprime = xprime[None, :, :]
        yprime = yprime[None, :, :]
        
        
        self.x = tf.Variable(xprime, name='x', trainable=False)
        self.y = tf.Variable(yprime, name='y', trainable=False)
        
        self.kernel = self.create_kernel()

In [ ]:
# export
@patch
def create_kernel(self:GaborLayer):
    ori_y_term = self.gammas ** 2 * self.y ** 2
    exponent_ori = (self.x ** 2 + ori_y_term) * self.sigmas ** 2
    gaussian_term_ori = tf.exp(-exponent_ori)
    cosine_term = tf.cos(self.x * self.lambdas + self.psis)
    ori_gb = gaussian_term_ori * cosine_term
    self.ori_gb = ori_gb
    ori_gb = tf.reshape(ori_gb, 
                        (self.filters * self.orientations, 
                         *self.kernel_size))
    
    if self.use_alphas:
        ori_gb = self.alphas * ori_gb
    
    return ori_gb

In [ ]:
GL = GaborLayer(filters=2, kernel_size=(15, 15), orientations=8)
GL.build([None, 28, 28, 3])